In [3]:
import ta

import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.instruments import USD, BTC, ETH, LTC
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order

In [49]:
cdd = CryptoDataDownload()

bitfinex_data = pd.concat([
    cdd.fetch("Bitfinex", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitfinex", "USD", "ETH", "1h").add_prefix("ETH:")
], axis=1)

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitstamp", "USD", "LTC", "1h").add_prefix("LTC:")
], axis=1)


In [10]:
bitfinex_data['BTC:date'].min()

Timestamp('2018-05-15 06:00:00')

In [12]:
bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(list(bitfinex_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitfinex_data['ETH:close']), dtype="float").rename("USD-ETH")
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitstamp_data['LTC:close']), dtype="float").rename("USD-LTC")
)

In [14]:
# Add all features for bitstamp BTC & ETH
bitfinex_btc = bitfinex_data.loc[:, [name.startswith("BTC") for name in bitfinex_data.columns]]
bitfinex_eth = bitfinex_data.loc[:, [name.startswith("ETH") for name in bitfinex_data.columns]]


In [15]:
bitfinex_btc

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume
0,2018-05-15 06:00:00,1526364000,8723.8,8793.0,8714.9,8739.0,8.988054e+06
1,2018-05-15 07:00:00,1526367600,8739.0,8754.8,8719.3,8743.0,2.288904e+06
2,2018-05-15 08:00:00,1526371200,8743.0,8743.1,8653.2,8723.7,8.891773e+06
3,2018-05-15 09:00:00,1526374800,8723.7,8737.8,8701.2,8708.1,2.054868e+06
4,2018-05-15 10:00:00,1526378400,8708.1,8855.7,8695.8,8784.4,1.730972e+07
...,...,...,...,...,...,...,...
50812,2024-03-01 21:00:00,1709326800000,62756.0,62809.0,62204.0,62516.0,1.526886e+07
50813,2024-03-01 22:00:00,1709330400000,62500.0,62638.0,62281.0,62477.0,3.370639e+06
50814,2024-03-01 23:00:00,1709334000000,62470.0,62703.0,62319.0,62348.0,2.883944e+06
50815,2024-03-02 00:00:00,1709337600000,62330.0,62406.0,62054.0,62259.0,1.983829e+06


In [16]:
with NameSpace("bitfinex"):
    bitfinex_streams = [
        Stream.source(list(bitfinex_btc[c]), dtype="float").rename(c) for c in bitfinex_btc.columns
    ]
    bitfinex_streams += [
        Stream.source(list(bitfinex_eth[c]), dtype="float").rename(c) for c in bitfinex_eth.columns
    ]

In [17]:
# Add all features for bitstamp BTC & LTC
bitstamp_btc = bitstamp_data.loc[:, [name.startswith("BTC") for name in bitstamp_data.columns]]  
bitstamp_ltc = bitstamp_data.loc[:, [name.startswith("LTC") for name in bitstamp_data.columns]]


In [18]:
ta.add_all_ta_features(
    bitstamp_ltc,
    colprefix="LTC:",
    **{k: "LTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

C:\Users\thiago\AppData\Roaming\Python\Python311\site-packages\ta\wrapper.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_adi"] = AccDistIndexIndicator(
C:\Users\thiago\AppData\Roaming\Python\Python311\site-packages\ta\wrapper.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_obv"] = OnBalanceVolumeIndicator(
C:\Users\thiago\AppData\Roaming\Python\Python311\site-packages\ta\wrapper.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,LTC:date,LTC:unix,LTC:open,LTC:high,LTC:low,LTC:close,LTC:volume,LTC:volume_adi,LTC:volume_obv,LTC:volume_cmf,...,LTC:momentum_ppo,LTC:momentum_ppo_signal,LTC:momentum_ppo_hist,LTC:momentum_pvo,LTC:momentum_pvo_signal,LTC:momentum_pvo_hist,LTC:momentum_kama,LTC:others_dr,LTC:others_dlr,LTC:others_cr
0,2018-05-15 06:00:00,1526364000,147.20,148.70,147.01,147.20,1907.280000,-1.478424e+03,1.907280e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2018-05-15 07:00:00,1526367600,147.20,148.02,146.80,147.50,557.940000,-1.396105e+03,2.465220e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.203804,0.203597,0.203804
2,2018-05-15 08:00:00,1526371200,147.50,147.50,144.54,145.64,3606.380000,-2.322068e+03,-1.141160e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.261017,-1.269035,-1.059783
3,2018-05-15 09:00:00,1526374800,145.64,146.62,145.15,145.54,369.670000,-2.495586e+03,-1.510830e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.068662,-0.068686,-1.127717
4,2018-05-15 10:00:00,1526378400,145.54,149.14,144.95,147.66,2755.480000,-1.686698e+03,1.244650e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.456644,1.446137,0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50846,2024-03-02 20:00:00,1709409600,91.94,92.81,91.06,91.86,54949.724492,3.650237e+08,1.558049e+08,-0.010105,...,1.835145,1.962026,-0.126882,-28.903481,-15.787263,-13.116217,90.656209,-0.119604,-0.119676,-37.595109
50847,2024-03-02 21:00:00,1709413200,91.84,92.41,91.84,92.24,43962.189058,3.650415e+08,1.558489e+08,0.042108,...,1.797900,1.929201,-0.131301,-32.746986,-19.179208,-13.567778,90.760290,0.413673,0.412820,-37.336957
50848,2024-03-02 22:00:00,1709416800,92.04,92.21,91.37,91.84,58516.913595,3.650485e+08,1.557904e+08,-0.003044,...,1.712863,1.885933,-0.173070,-35.255746,-22.394515,-12.861231,90.772512,-0.433651,-0.434594,-37.608696
50849,2024-03-02 23:00:00,1709420400,92.08,94.60,91.87,94.60,259490.208140,3.653079e+08,1.560499e+08,-0.053711,...,1.867031,1.882153,-0.015122,-26.405293,-23.196671,-3.208622,91.178271,3.005226,2.960954,-35.733696


In [19]:
bitstamp_ltc

,LTC:date,LTC:unix,LTC:open,LTC:high,LTC:low,LTC:close,LTC:volume,LTC:volume_adi,LTC:volume_obv,LTC:volume_cmf,...,LTC:momentum_ppo,LTC:momentum_ppo_signal,LTC:momentum_ppo_hist,LTC:momentum_pvo,LTC:momentum_pvo_signal,LTC:momentum_pvo_hist,LTC:momentum_kama,LTC:others_dr,LTC:others_dlr,LTC:others_cr
0,2018-05-15 06:00:00,1526364000,147.20,148.70,147.01,147.20,1907.280000,-1.478424e+03,1.907280e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2018-05-15 07:00:00,1526367600,147.20,148.02,146.80,147.50,557.940000,-1.396105e+03,2.465220e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.203804,0.203597,0.203804
2,2018-05-15 08:00:00,1526371200,147.50,147.50,144.54,145.64,3606.380000,-2.322068e+03,-1.141160e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.261017,-1.269035,-1.059783
3,2018-05-15 09:00:00,1526374800,145.64,146.62,145.15,145.54,369.670000,-2.495586e+03,-1.510830e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.068662,-0.068686,-1.127717
4,2018-05-15 10:00:00,1526378400,145.54,149.14,144.95,147.66,2755.480000,-1.686698e+03,1.244650e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.456644,1.446137,0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50846,2024-03-02 20:00:00,1709409600,91.94,92.81,91.06,91.86,54949.724492,3.650237e+08,1.558049e+08,-0.010105,...,1.835145,1.962026,-0.126882,-28.903481,-15.787263,-13.116217,90.656209,-0.119604,-0.119676,-37.595109
50847,2024-03-02 21:00:00,1709413200,91.84,92.41,91.84,92.24,43962.189058,3.650415e+08,1.558489e+08,0.042108,...,1.797900,1.929201,-0.131301,-32.746986,-19.179208,-13.567778,90.760290,0.413673,0.412820,-37.336957
50848,2024-03-02 22:00:00,1709416800,92.04,92.21,91.37,91.84,58516.913595,3.650485e+08,1.557904e+08,-0.003044,...,1.712863,1.885933,-0.173070,-35.255746,-22.394515,-12.861231,90.772512,-0.433651,-0.434594,-37.608696
50849,2024-03-02 23:00:00,1709420400,92.08,94.60,91.87,94.60,259490.208140,3.653079e+08,1.560499e+08,-0.053711,...,1.867031,1.882153,-0.015122,-26.405293,-23.196671,-3.208622,91.178271,3.005226,2.960954,-35.733696


In [20]:
with NameSpace("bitfinex"):
    bitfinex_streams = [
        Stream.source(list(bitfinex_btc[c]), dtype="float").rename(c) for c in bitfinex_btc.columns
    ]
    bitfinex_streams += [
        Stream.source(list(bitfinex_eth[c]), dtype="float").rename(c) for c in bitfinex_eth.columns
    ]


In [23]:
# Add all features for bitstamp BTC & LTC
bitstamp_btc = bitstamp_data.loc[:, [name.startswith("BTC") for name in bitstamp_data.columns]]
bitstamp_ltc = bitstamp_data.loc[:, [name.startswith("LTC") for name in bitstamp_data.columns]]

In [24]:
ta.add_all_ta_features(
    bitstamp_ltc,
    colprefix="LTC:",
    **{k: "LTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

C:\Users\thiago\AppData\Roaming\Python\Python311\site-packages\ta\wrapper.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_adi"] = AccDistIndexIndicator(
C:\Users\thiago\AppData\Roaming\Python\Python311\site-packages\ta\wrapper.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_obv"] = OnBalanceVolumeIndicator(
C:\Users\thiago\AppData\Roaming\Python\Python311\site-packages\ta\wrapper.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,LTC:date,LTC:unix,LTC:open,LTC:high,LTC:low,LTC:close,LTC:volume,LTC:volume_adi,LTC:volume_obv,LTC:volume_cmf,...,LTC:momentum_ppo,LTC:momentum_ppo_signal,LTC:momentum_ppo_hist,LTC:momentum_pvo,LTC:momentum_pvo_signal,LTC:momentum_pvo_hist,LTC:momentum_kama,LTC:others_dr,LTC:others_dlr,LTC:others_cr
0,2018-05-15 06:00:00,1526364000,147.20,148.70,147.01,147.20,1907.280000,-1.478424e+03,1.907280e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2018-05-15 07:00:00,1526367600,147.20,148.02,146.80,147.50,557.940000,-1.396105e+03,2.465220e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.203804,0.203597,0.203804
2,2018-05-15 08:00:00,1526371200,147.50,147.50,144.54,145.64,3606.380000,-2.322068e+03,-1.141160e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.261017,-1.269035,-1.059783
3,2018-05-15 09:00:00,1526374800,145.64,146.62,145.15,145.54,369.670000,-2.495586e+03,-1.510830e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.068662,-0.068686,-1.127717
4,2018-05-15 10:00:00,1526378400,145.54,149.14,144.95,147.66,2755.480000,-1.686698e+03,1.244650e+03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.456644,1.446137,0.312500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50846,2024-03-02 20:00:00,1709409600,91.94,92.81,91.06,91.86,54949.724492,3.650237e+08,1.558049e+08,-0.010105,...,1.835145,1.962026,-0.126882,-28.903481,-15.787263,-13.116217,90.656209,-0.119604,-0.119676,-37.595109
50847,2024-03-02 21:00:00,1709413200,91.84,92.41,91.84,92.24,43962.189058,3.650415e+08,1.558489e+08,0.042108,...,1.797900,1.929201,-0.131301,-32.746986,-19.179208,-13.567778,90.760290,0.413673,0.412820,-37.336957
50848,2024-03-02 22:00:00,1709416800,92.04,92.21,91.37,91.84,58516.913595,3.650485e+08,1.557904e+08,-0.003044,...,1.712863,1.885933,-0.173070,-35.255746,-22.394515,-12.861231,90.772512,-0.433651,-0.434594,-37.608696
50849,2024-03-02 23:00:00,1709420400,92.08,94.60,91.87,94.60,259490.208140,3.653079e+08,1.560499e+08,-0.053711,...,1.867031,1.882153,-0.015122,-26.405293,-23.196671,-3.208622,91.178271,3.005226,2.960954,-35.733696


In [26]:
with NameSpace("bitstamp"):
    bitstamp_streams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    bitstamp_streams += [
        Stream.source(list(bitstamp_ltc[c]), dtype="float").rename(c) for c in bitstamp_ltc.columns
    ]
    
feed = DataFeed(bitfinex_streams + bitstamp_streams)

In [39]:
feed.next()

{'bitfinex:/BTC:date': Timestamp('2018-05-15 18:00:00'),
 'bitfinex:/BTC:unix': 1526407200,
 'bitfinex:/BTC:open': 8553.8,
 'bitfinex:/BTC:high': 8553.8,
 'bitfinex:/BTC:low': 8438.1,
 'bitfinex:/BTC:close': 8516.5,
 'bitfinex:/BTC:volume': 23290913.96,
 'bitfinex:/ETH:date': Timestamp('2018-05-15 18:00:00'),
 'bitfinex:/ETH:unix': 1526407200.0,
 'bitfinex:/ETH:open': 716.54,
 'bitfinex:/ETH:high': 716.54,
 'bitfinex:/ETH:low': 707.4,
 'bitfinex:/ETH:close': 712.61,
 'bitfinex:/ETH:volume': 7953979.54,
 'bitstamp:/BTC:date': Timestamp('2018-05-15 18:00:00'),
 'bitstamp:/BTC:unix': 1526407200,
 'bitstamp:/BTC:open': 8546.34,
 'bitstamp:/BTC:high': 8555.51,
 'bitstamp:/BTC:low': 8400.0,
 'bitstamp:/BTC:close': 8526.65,
 'bitstamp:/BTC:volume': 877.26,
 'bitstamp:/LTC:date': Timestamp('2018-05-15 18:00:00'),
 'bitstamp:/LTC:unix': 1526407200,
 'bitstamp:/LTC:open': 140.95,
 'bitstamp:/LTC:high': 140.95,
 'bitstamp:/LTC:low': 138.8,
 'bitstamp:/LTC:close': 140.54,
 'bitstamp:/LTC:volume': 

In [40]:
portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC),
    Wallet(bitfinex, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 3 * LTC),
])

env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="simple",
    feed=feed,
    window_size=15,
    enable_logger=False
)

In [41]:
env.observer.feed.next()


{'internal': {'bitfinex:/USD-BTC': 8739.0,
  'bitfinex:/USD-ETH': 733.04,
  'bitfinex:/USD:/free': 10000.0,
  'bitfinex:/USD:/locked': 0.0,
  'bitfinex:/USD:/total': 10000.0,
  'bitfinex:/BTC:/free': 10.0,
  'bitfinex:/BTC:/locked': 0.0,
  'bitfinex:/BTC:/total': 10.0,
  'bitfinex:/BTC:/worth': 87390.0,
  'bitfinex:/ETH:/free': 5.0,
  'bitfinex:/ETH:/locked': 0.0,
  'bitfinex:/ETH:/total': 5.0,
  'bitfinex:/ETH:/worth': 3665.2,
  'bitstamp:/USD-BTC': 8740.99,
  'bitstamp:/USD-LTC': 147.2,
  'bitstamp:/USD:/free': 1000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 1000.0,
  'bitstamp:/BTC:/free': 5.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 5.0,
  'bitstamp:/BTC:/worth': 43704.95,
  'bitstamp:/LTC:/free': 3.0,
  'bitstamp:/LTC:/locked': 0.0,
  'bitstamp:/LTC:/total': 3.0,
  'bitstamp:/LTC:/worth': 441.59999999999997,
  'net_worth': 146201.75},
 'external': {'bitfinex:/BTC:date': Timestamp('2018-05-15 06:00:00'),
  'bitfinex:/BTC:unix': 1526364000,
  'bit